# 3 Pre-Processing and Training Data<a id='4_Pre-Processing_and_Training_Data'></a>

The objectives of this notebook is to create a machine learning capable to segmentize a given Tweet into its category. We will first use an unsupervise learning method called K-Means define the clusters of the subjectivity of the Tweets gathered in notebook 01_Data_Wrangling, then measure the overall sentiments of all Tweets in a given day to predict the closing price.


Guidance from Springboard:
* Create dummy or indicator features for categorical variables
* Standardize the magnitude of numeric features using a scaler
* Split your data into testing and training datasets

## 3.1 Imports<a id='4.3_Imports'></a>

In [3]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime

from library.sb_utils import save_file

ModuleNotFoundError: No module named 'sb_utils'